In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import  collections

In [2]:
df=pd.read_csv('Locality_crime.csv') 

In [28]:
df_weather = pd.read_csv('suburb_weather.csv',header=None)

In [4]:
# choose observations only with property crime 
options = ['Dwelling Burglary', 'Non-Dwelling Burglary', 'Stealing of Motor Vehicle', 'Stealing', 'Property Damage', 'Arson']
prop_df = df[df['Website Category Names'].isin(options)]

In [5]:
# remove Annual and Year columns 
df = prop_df.drop('Annual', 1)

In [6]:
df = df.drop("Year", 1)

In [10]:
# group data by both 'Suburb' and 'Crime type'
df.head()

,Suburb,Website Category Names,Jul,Aug,Sep,Oct,Nov,Dec,Jan,Feb,Mar,Apr,May,Jun
1,Abba River,Non-Dwelling Burglary,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0
2,Abba River,Stealing,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0
3,Abba River,Stealing,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0
4,Abba River,Stealing,0,0,0,0,0,1,0,1,0,0.0,0.0,0.0
5,Abba River,Stealing of Motor Vehicle,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0


In [8]:
# convert factor months to observation (row to column)
df2 = pd.melt(df, id_vars=["Suburb", "Website Category Names"], var_name="Month", value_name="Count")

In [9]:
df2.head()

,Suburb,Website Category Names,Month,Count
0,Abba River,Non-Dwelling Burglary,Jul,0.0
1,Abba River,Stealing,Jul,0.0
2,Abba River,Stealing,Jul,0.0
3,Abba River,Stealing,Jul,0.0
4,Abba River,Stealing of Motor Vehicle,Jul,0.0


In [11]:
# group data by sum  
df_group = df2.groupby(['Suburb', 'Website Category Names', 'Month']).sum()

In [12]:
df_group = df_group.reset_index()

In [13]:
df_group[:5]

,Suburb,Website Category Names,Month,Count
0,Abba River,Non-Dwelling Burglary,Apr,1.0
1,Abba River,Non-Dwelling Burglary,Aug,0.0
2,Abba River,Non-Dwelling Burglary,Dec,0.0
3,Abba River,Non-Dwelling Burglary,Feb,0.0
4,Abba River,Non-Dwelling Burglary,Jan,0.0


In [30]:
# Unnamed: 0	Unnamed: 0.1	Month	AvgMinTemp	AvgMaxTemp	AvgDailyRainfall	SuburbName
# change header in weather data
df_weather.columns = ['Id','Month_id','Month','AvgMinTemp','AvgMaxTemp','AvgDailyRainfall','SuburbName']

In [31]:
df_weather.head()

,Id,Month_id,Month,AvgMinTemp,AvgMaxTemp,AvgDailyRainfall,SuburbName
0,0,0,January,21.7,32.741936,1.50,Rockwell
1,1,1,February,22.7,32.925000,1.16,Rockwell
2,2,2,March,21.9,31.757862,0.67,Rockwell
3,3,3,April,19.8,29.020000,0.85,Rockwell
4,4,4,May,16.9,25.025806,1.61,Rockwell


In [18]:
#remove first row of weather data (header remove)
#df_weather = df_weather.iloc[1: , :]

In [32]:
df_weather[:5]

,Id,Month_id,Month,AvgMinTemp,AvgMaxTemp,AvgDailyRainfall,SuburbName
0,0,0,January,21.7,32.741936,1.50,Rockwell
1,1,1,February,22.7,32.925000,1.16,Rockwell
2,2,2,March,21.9,31.757862,0.67,Rockwell
3,3,3,April,19.8,29.020000,0.85,Rockwell
4,4,4,May,16.9,25.025806,1.61,Rockwell


In [33]:
# suburbs = list(set(list(df_group['Suburb'])))
suburbs_unique = []
for suburb in df_group['Suburb']:
    if suburb not in suburbs_unique:
        suburbs_unique.append(suburb)
len(suburbs_unique)

1588

In [34]:
# change 'Month' to numbers that same with month 'Month_id' in weather data
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
target = [i for i in range(0,12)]
for m,t in zip(months,target):
    df_group['Month'] = np.where(df_group.Month == m,t,df_group.Month)


In [35]:
# merger two dataframe by Month_id and Surburb
suburb_weather_months_data = collections.defaultdict(list)
for suburb in suburbs_unique:
    # current suburb
    current_suburb = df_group[df_group.Suburb == suburb]
    # the weather of current suburb
    current_suburb_weather = df_weather[df_weather.SuburbName == suburb]
    # the months
    suburb_weather_months = list(current_suburb['Month'])
    for m in suburb_weather_months:
        suburb_weather_months_data['AvgMinTemp'].append(float(current_suburb_weather[current_suburb_weather['Month_id'] == m]['AvgMinTemp']))
        suburb_weather_months_data['AvgMaxTemp'].append(float(current_suburb_weather[current_suburb_weather['Month_id'] == m]['AvgMaxTemp']))
        suburb_weather_months_data['AvgDailyRainfall'].append(float(current_suburb_weather[current_suburb_weather['Month_id'] == m]['AvgDailyRainfall']))
    # current_suburb['AvgMinTemp'] = suburb_weather_months_data['AvgMinTemp']
    # current_suburb['AvgMaxTemp'] = suburb_weather_months_data['AvgMaxTemp']
    # current_suburb['AvgDailyRainfall'] = suburb_weather_months_data['AvgDailyRainfall']


In [36]:
    df_group['AvgMinTemp'] = suburb_weather_months_data['AvgMinTemp']
    df_group['AvgMaxTemp'] = suburb_weather_months_data['AvgMaxTemp']
    df_group['AvgDailyRainfall'] = suburb_weather_months_data['AvgDailyRainfall']

In [37]:
# the final mergerd data (weather and surburb)
df_group

,Suburb,Website Category Names,Month,Count,AvgMinTemp,AvgMaxTemp,AvgDailyRainfall
0,Abba River,Non-Dwelling Burglary,3,1.0,16.1,30.695667,0.34
1,Abba River,Non-Dwelling Burglary,7,0.0,9.7,26.951612,0.19
2,Abba River,Non-Dwelling Burglary,11,0.0,23.7,39.006450,2.25
3,Abba River,Non-Dwelling Burglary,1,0.0,23.2,36.046715,2.36
4,Abba River,Non-Dwelling Burglary,0,0.0,24.7,39.706450,2.38
...,...,...,...,...,...,...,...
85387,Zanthus,Property Damage,2,1.0,17.4,30.203226,1.58
85388,Zanthus,Property Damage,4,0.0,11.1,20.677420,0.67
85389,Zanthus,Property Damage,10,0.0,15.2,30.746666,1.20
85390,Zanthus,Property Damage,9,0.0,13.1,29.300259,0.99


In [94]:
#trandfer data to csv file
df_group.to_csv('new_data_with_temp.csv')

In [38]:
df_weather[df_weather.SuburbName=='Zanthus']

,Id,Month_id,Month,AvgMinTemp,AvgMaxTemp,AvgDailyRainfall,SuburbName
6024,672,0,January,19.2,35.580647,1.90,Zanthus
6025,673,1,February,19.1,35.266537,1.30,Zanthus
6026,674,2,March,17.4,30.203226,1.58,Zanthus
6027,675,3,April,15.2,27.100000,0.80,Zanthus
6028,676,4,May,11.1,20.677420,0.67,Zanthus
6029,677,5,June,9.0,18.363333,0.88,Zanthus
6030,678,6,July,8.0,17.741936,0.84,Zanthus
6031,679,7,August,8.8,19.938710,0.87,Zanthus
6032,680,8,September,10.0,24.396667,0.70,Zanthus
6033,681,9,October,13.1,29.300259,0.99,Zanthus
